# Customer's error function

To the error function it is expected the following pandas.DataFrame struncture:

| StoreID  | Month | Region | NumberOfSales | _NumberOfSales |
| ---------|----------------|---------------| ---------------|
| 1000	   |3      | 4      | 16            | 16             |
| 1000	   |4      | 4      | 30            | 23             |
| 1001	   |3      | 6      | 410           | 411            |
| 1001	   |4      | 27     | 3130          | 3120           |
| 1002	   |3      | 58     | 10            | 8              |

Where:
 
 - *NumberOfSales* are the test values
 - *_NumberOfSales* are the predicted values
 
 

Start from test set to simulate a predicted dataset

In [17]:
from import_man import *

error_evaluation_columns = ['StoreID', 'Month', 'Region', 'NumberOfSales', '_NumberOfSales']

df = pd.read_csv('./dataset/train.csv')

print("Shape before: " + str(df.shape))

# Let's work on a reduced instance of the test set
df = df.sample(n=5000)

print("Shape after: " + str(df.shape))

Shape before: (523021, 36)
Shape after: (5000, 36)


In [18]:
from datetime import datetime

# Parse the month and add it as attribute
df['Month'] =  df['Date'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y').month)
df.head()

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,AssortmentType,NearestCompetitor,Region,NumberOfCustomers,...,Mean_VisibilityKm,Mean_Wind_SpeedKm_h,Min_Dew_PointC,Min_Humidity,Min_Sea_Level_PressurehPa,Min_TemperatureC,Min_VisibilitykM,Precipitationmm,WindDirDegrees,Month
63289,1090,12/11/2016,0,1,0,Standard Market,General,18551,10,312,...,10.0,29,3,62,999,6,10.0,0.00,231,11
141449,1203,27/05/2016,0,1,0,Hyper Market,With Non-Food Department,21325,7,197,...,13.0,8,-3,26,1010,3,10.0,0.00,214,5
218866,1312,10/05/2017,0,1,1,Standard Market,With Non-Food Department,1963,7,175,...,11.0,18,6,37,1009,11,10.0,1.02,232,5
266214,1380,24/04/2017,1,0,0,Hyper Market,General,224,9,0,...,13.0,11,5,47,1003,5,10.0,0.00,47,4
190227,1271,02/04/2017,0,0,0,Super Market,General,1583,7,0,...,22.0,8,2,20,1014,4,18.0,0.00,49,4


In [19]:
# Create fake predicted sales 
df['_NumberOfSales'] = df.NumberOfSales.apply(lambda x: df['NumberOfSales'].sample().values[0])

# Remove useless columns
df =  df[error_evaluation_columns]

df.head(20)

,StoreID,Month,Region,NumberOfSales,_NumberOfSales
63289,1090,11,10,5290,8461
141449,1203,5,7,2346,0
218866,1312,5,7,4322,9120
266214,1380,4,9,0,0
190227,1271,4,7,0,3706
177999,1254,3,2,5269,10221
100571,1145,3,10,0,3443
343099,1490,11,0,3223,3933
168609,1240,12,4,0,4149
13468,1018,2,4,2650,2038


In [20]:
df_sums_by_region = df.groupby(['Region']).sum()

df_sums_by_region['abs_a_minus_p'] = abs(df_sums_by_region['NumberOfSales'] - df_sums_by_region['_NumberOfSales'])

df_sums_by_region.head(20)

,StoreID,Month,NumberOfSales,_NumberOfSales,abs_a_minus_p
Region,,,,,
0,520240,2362,1479641,1498608,18967
1,228972,1135,701142,650596,50546
2,953631,3970,2651072,2867312,216240
3,892999,4426,3062579,2644517,418062
4,153879,754,519385,417370,102015
5,469270,2338,1333972,1508967,174995
6,348560,1624,822998,996546,173548
7,581622,2891,1669821,1757499,87678
8,243128,1191,638279,694936,56657


In [21]:
# Total error per region
df_sums_by_region['E_r'] = df_sums_by_region['abs_a_minus_p'] / df_sums_by_region['NumberOfSales']

df_sums_by_region.head(20)

,StoreID,Month,NumberOfSales,_NumberOfSales,abs_a_minus_p,E_r
Region,,,,,,
0,520240,2362,1479641,1498608,18967,0.012819
1,228972,1135,701142,650596,50546,0.072091
2,953631,3970,2651072,2867312,216240,0.081567
3,892999,4426,3062579,2644517,418062,0.136507
4,153879,754,519385,417370,102015,0.196415
5,469270,2338,1333972,1508967,174995,0.131183
6,348560,1624,822998,996546,173548,0.210873
7,581622,2891,1669821,1757499,87678,0.052507
8,243128,1191,638279,694936,56657,0.088765


In [22]:
# Get the number of regions
N_regions = len(df.Region.unique())

print("Number of regions: {}".format(N_regions))

Number of regions: 11


In [23]:
Error = df_sums_by_region['E_r'].sum() / N_regions

print("Total error: {}".format(Error))

Total error: 0.09097740346076945
